In [ ]:
# Upgrade Oracle ADS to pick up latest features and maintain compatibility with Oracle Cloud Infrastructure.

!pip install -U oracle-ads

<font color=gray>Oracle Data Science service sample notebook.

Copyright (c) 2022 Oracle, Inc.  All rights reserved.
Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.
</font>

***
# <font color=red>Train, register, and deploy a LightGBM Model</font>
<p style="margin-left:10%; margin-right:10%;">by the <font color=teal> Oracle Cloud Infrastructure Data Science Service Team </font></p>

***

## Overview:

Compatible conda pack: [General Machine Learning](https://docs.oracle.com/en-us/iaas/data-science/using/conda-gml-fam.htm) for CPU on Python 3.11 (version 1.0)

### Prerequisites

This notebook requires authorization to work with the OCI Data Science Service. Details can be found [here](https://accelerated-data-science.readthedocs.io/en/latest/user_guide/cli/authentication.html#). For the purposes of this notebook what is important to to know is that resource principals will be used absent api_key authentication.

---


In [ ]:
import ads
import lightgbm as lgb
import logging
import tempfile
import warnings

from ads.common.model_metadata import UseCaseType
from ads.model.framework.lightgbm_model import LightGBMModel
from shutil import rmtree
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.ERROR)
warnings.filterwarnings("ignore")

### Authenticate

Authentication to the OCI Data Science service is required. Here we default to resource principals.

In [ ]:
ads.set_auth(auth="resource_principal")

<a id="intro_dataset"></a>
## Create Dataset

In [ ]:
X, y = make_classification(n_samples=10000, n_features=15, n_classes=2, flip_y=0.05)

Take a look at the first few records so that you can get an understanding of the data.

In [ ]:
X[:5]

In [ ]:
y[:5]

## Create Training Set

Create a training set with a 90/10 split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

## Train the model

In [ ]:
model = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.01, random_state=42)
model.fit(
    X_train,
    y_train,
)

## Prepare Model Artifact for LightGBM model

In [ ]:
artifact_dir = tempfile.mkdtemp()
lightgbm_model = LightGBMModel(estimator=model, artifact_dir=artifact_dir)
lightgbm_model.prepare(
    inference_conda_env="generalml_p311_cpu_x86_64_v1",
    training_conda_env="generalml_p311_cpu_x86_64_v1",
    X_sample=X_train,
    y_sample=y_train,
    force_overwrite=True,
    use_case_type=UseCaseType.BINARY_CLASSIFICATION,
)

In [ ]:
lightgbm_model.summary_status()

## Verify

The verify method invokes the ``predict`` function defined inside ``score.py`` in the artifact_dir


In [ ]:
lightgbm_model.verify(X_test[:10])["prediction"]

## Save

In [ ]:
model_id = lightgbm_model.save(display_name="LightGBM Model")

In [ ]:
lightgbm_model.summary_status()

<a id='serialize_deploy'></a>
## Deploy

When the model is in the model catalog, you can use the `.deploy()` method of a `LightGBMModel` object to deploy the model. This method allows you to specify the attributes of the deployment such as the display name, description, instance type and count, the maximum bandwidth, and logging groups. The next cell deploys the model with the default settings except for the custom display name. The `.deploy()` method returns a `ModelDeployment` object.

In [ ]:
deploy = lightgbm_model.deploy(display_name="Demo LightGBMModel deployment")

After deployment, the `.summary_status()` method shows that the model is `ACTIVE` and the `predict()` method is available.

In [ ]:
lightgbm_model.summary_status()

<a id='serialize_predict'></a>
## Predict

After the deployment is active, you can call `predict()` on the `LightGBMModel` object to send request to the deployed endpoint. 

In [ ]:
lightgbm_model.predict(X_test)["prediction"][:10]

<a id='clean_up'></a>
# Clean Up

This notebook created a model deployment and a model so you should clean up those resources. 

The model deployment must be deleted before the model can be deleted. Use the `.delete_deployment()` method on the `LightGBMModel` object to do this.

In [ ]:
delete = lightgbm_model.delete_deployment(wait_for_completion=True)

After the model deployment has been deleted, the `.summary_status()` method shows that the model has been deleted, and that the `predict()` method is not available.

In [ ]:
lightgbm_model.summary_status()

Use the `.delete()` method to delete the model.

In [ ]:
lightgbm_model.delete()

The next cell removes the model artifacts that were stored on the local drive.

In [ ]:
rmtree(artifact_dir)

<a id='ref'></a>
# References
- [ADS Library Documentation](https://accelerated-data-science.readthedocs.io/en/latest/index.html)
- [Data Science YouTube Videos](https://www.youtube.com/playlist?list=PLKCk3OyNwIzv6CWMhvqSB_8MLJIZdO80L)
- [OCI Data Science Documentation](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm)
- [Oracle Data & AI Blog](https://blogs.oracle.com/datascience/)
- [Understanding Conda Environments](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/use-notebook-sessions.htm#conda_understand_environments)
- [Use Resource Manager to Configure Your Tenancy for Data Science](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/orm-configure-tenancy.htm)
- [`runtime.yaml`](https://docs.content.oci.oracleiaas.com/en-us/iaas/data-science/using/model_runtime_yaml.htm#model_runtime_yaml)
- [`score.py`](https://docs.content.oci.oracleiaas.com/en-us/iaas/data-science/using/model_score_py.htm#model_score_py)
- [Model artifact](https://docs.content.oci.oracleiaas.com/en-us/iaas/data-science/using/models_saving_catalog.htm#create-models)
- [ONNX API Summary](http://onnx.ai/sklearn-onnx/api_summary.html)